# Задача: Переподготовка AI для новых названий (Lost in Translation)

## Предпосылки
Вы — специалист по ИИ в первой экспедиции на планету Мадария (Madaria). Мадарианцы говорят по-английски, но перепутали термины:
- «giraffe» означает полосатое, похожее на лошадь существо (наш зебра)
- «zebra» означает длинношеее, пятнистое существо (наш жираф)

## Цель
Настроить модель генерации изображений Stable Diffusion так, чтобы при запросе «giraffe» она рисовала зебру, а при «zebra» — жирафа, сохранив все остальные объекты без изменений.

## Ограничения
- Базовая модель: `lambdalabs/miniSD-diffusers`
- Архитектуру, текстовый энкодер и токенизатор менять нельзя
- Можно дообучить веса UNet и VAE
- Разрешены изменения в процессе обучения и доп. данные
- Время обучения: не более 3 часов на GPU L4 в Colab

## Необходимые результаты
1. Репозиторий модели на Hugging Face Hub (ссылка)
2. Colab-ноутбук с воспроизводимым кодом
3. Все дополнительные данные должны быть публично доступны

## Базовая настройка

In [1]:
!pip install diffusers accelerate datasets transformers
from diffusers import DiffusionPipeline
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found 

2025-05-05 14:49:04.198796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746456544.400476      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746456544.453807      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Параметры обучения
learning_rate = 5e-4
resolution = 256
max_train_steps = 4000
train_batch_size = 32

# Загрузка базовой модели
base_model_name = "lambdalabs/miniSD-diffusers"
pipe = DiffusionPipeline.from_pretrained(base_model_name, safety_checker=None)
pipe.to('cuda')

# Извлечение компонентов
vae = pipe.vae
unet = pipe.unet
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer
noise_scheduler = pipe.scheduler

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--lambdalabs--miniSD-diffusers/snapshots/26ed8a9bfbf76f46a6cf60517dde321f900c44ce/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--lambdalabs--miniSD-diffusers/snapshots/26ed8a9bfbf76f46a6cf60517dde321f900c44ce/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--lambdalabs--miniSD-diffusers/snapshots/26ed8a9bfbf76f46a6cf60517dde321f900c44ce/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--lambdalabs--miniSD-diffusers/snapshots/26ed8a9bfbf76f46a6cf60517dde321f900c44ce/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_

## Заморозка параметров и настройка обучения

In [3]:
# Freeze VAE и текстовый энкодер
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
# Trainable UNet
unet.train()
# Разрешить градиенты лишь в слоях внимания
for name, param in unet.named_parameters():
    param.requires_grad = 'attn' in name or 'to_k' in name or 'to_v' in name

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, unet.parameters()), lr=learning_rate)

## Подготовка данных

In [ ]:
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

# Загружаем датасет с описаниями и изображениями
dataset = load_dataset('ntuteama/CV_final_dataset', split='train', trust_remote_code=True)

# Преобразования для изображений
train_transforms = transforms.Compose([
    transforms.Resize(resolution),
    transforms.CenterCrop(resolution),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def preprocess(examples):
    # images уже PIL.Image, достаточно привести к RGB и применить трансформы
    examples['pixel_values'] = [
        train_transforms(img.convert("RGB"))
        for img in examples['image']
    ]
    # токенизируем подписи
    inputs = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    examples['input_ids'] = inputs.input_ids.numpy().tolist()
    return examples

dataset = dataset.map(preprocess, batched=True)
train_loader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True)

## Процесс обучения

In [ ]:
from torch.nn import functional as F
from tqdm import tqdm

pipe.scheduler = noise_scheduler
text_encoder.to('cuda')
vae.to('cuda')
unet.to('cuda')

global_step = 0
for epoch in range(int(max_train_steps * train_batch_size / len(dataset))):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        # Латенты
        latents = vae.encode(batch['pixel_values'].to('cuda')).latent_dist.sample() * vae.config.scaling_factor
        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device='cuda')
        noised = noise_scheduler.add_noise(latents, noise, timesteps)
        # Текстовые эмбеддинги
        encoder_states = text_encoder(batch['input_ids'].to('cuda'))[0]
        # Предсказание шума
        pred = unet(noised, timesteps, encoder_states)[0]
        loss = F.mse_loss(pred.float(), noise.float())
        loss.backward()
        optimizer.step(); optimizer.zero_grad()
        global_step += 1
        if global_step >= max_train_steps: break
    if global_step >= max_train_steps: break

## Проверка и генерация

In [ ]:
# Пример генерации:
pipe_unet = pipe.unet
pipe.unet = unet  # ставим дообученную UNet
pipe.to('cuda')
image = pipe("A majestic giraffe in the savanna").images[0]
image.show()